(hover_dir_pad_1, hover_dir_pad_2, hover_numeric_pad)

(5 options, 5 options, 11 options) ~ 275

5 possible actions 

total 1375 state, action combinations

initial state is (A, A, A)
let's say we have to type 456A

(A, A, A)
(A, A, 4) A
(A, A, 5) A
(A, A, 6) A
(A, A, A) A


what if we're in like

(<, ^, 5) and we hit >
(A, ^, 5) and we hit >


(<, ^, 5) and we hit A?

(A, ^, 5) and we hit A?
(^, A, 5) and we hit A?
(^, >, A) and we hit A?

(A, A, 5) and we hit A?
(^, A, A) and we hit A?

(A, A, A) and we hit A?

In [1]:
# AAAAAAAAAAA0
# AAAAAAAAAAA9

In [2]:
import numpy as np

In [3]:
# input = """029A
# 980A
# 179A
# 456A
# 379A"""

input = open("inputs/21").read()

parsed_input = [list(s) for s in input.split()]

In [4]:
parsed_input

[['1', '4', '0', 'A'],
 ['1', '7', '0', 'A'],
 ['1', '6', '9', 'A'],
 ['8', '0', '3', 'A'],
 ['1', '2', '9', 'A']]

In [5]:
keypresses = parsed_input[0]
keypresses

['1', '4', '0', 'A']

In [6]:
import itertools

directional_board = np.array(
    [
        [None, "^", "A"],
        ["<", "v", ">"],
    ]
)

numeric_board = np.array(
    [
        ["7", "8", "9"],
        ["4", "5", "6"],
        ["1", "2", "3"],
        [None, "0", "A"],
    ]
)

dirs = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}


def in_bounds(tuple, shape):
    return 0 <= tuple[0] < shape[0] and 0 <= tuple[1] < shape[1]


def add_tuple(t1, t2):
    return tuple(map(sum, zip(t1, t2)))


def make_transition_dict(board):
    d = {}

    for i, j in np.ndindex(board.shape):
        if board[i, j] is None:
            continue

        for dir in dirs:
            new_pos = add_tuple((i, j), dirs[dir])
            d[(board[i, j], dir)] = (
                board[new_pos] if in_bounds(new_pos, board.shape) else None
            )

    return d


def manhattan_distances(array_2d: np.ndarray) -> dict:
    """
    Given a 2D NumPy array, returns a dictionary where the keys are tuples
    of two non-None elements, and the values are the Manhattan distance
    between their positions in the array.

    Each pair is represented in both orders, e.g., (A, B) and (B, A).
    """
    non_none_indices = np.argwhere(array_2d != None)
    values = array_2d[non_none_indices[:, 0], non_none_indices[:, 1]]

    elements = list(zip(values, non_none_indices[:, 0], non_none_indices[:, 1]))

    distance_dict = {}
    for (val1, row1, col1), (val2, row2, col2) in itertools.combinations(elements, 2):
        distance = abs(row1 - row2) + abs(col1 - col2)
        distance_dict[(val1, val2)] = distance
        distance_dict[(val2, val1)] = distance  # Include both directions

    return distance_dict


numeric_transition_dict = make_transition_dict(numeric_board)
directional_transition_dict = make_transition_dict(directional_board)
numeric_ds = manhattan_distances(numeric_board)
directional_ds = manhattan_distances(directional_board)

In [7]:
def get_next_state(state, action):
    *dir_hovers, num_hover = state
    dir_hovers_with_action = [action] + dir_hovers

    first_non_A_idx = next(
        (i for i, x in enumerate(dir_hovers_with_action) if x != "A"), None
    )
    # print(first_non_A_idx)

    # print(dir_hovers_with_action, first_non_A_idx)

    if first_non_A_idx is None:
        # print("emitting")
        # in this case we're emitting
        return state, num_hover

    if first_non_A_idx == len(dir_hovers_with_action) - 1:
        # print("modify num hover")
        # we modify the num_hover state
        new_num_hover = numeric_transition_dict[num_hover, dir_hovers_with_action[-1]]

        if not new_num_hover:
            return None

        return (*dir_hovers, new_num_hover), None
    else:
        # print("modify dir hover")
        # we modify the dir_hovers state
        new_dir_hovers_with_action_state = dir_hovers_with_action.copy()

        new_dir_state = directional_transition_dict[
            new_dir_hovers_with_action_state[first_non_A_idx + 1],
            new_dir_hovers_with_action_state[first_non_A_idx],
        ]

        if not new_dir_state:
            return None

        new_dir_hovers_with_action_state[first_non_A_idx + 1] = new_dir_state
        return (*new_dir_hovers_with_action_state[1:], num_hover), None

In [8]:
def get_possible_neighbors(state):
    for action in list(dirs.keys()) + ["A"]:
        if result := get_next_state(state, action):
            # state, emit = result
            # print(action, result)
            yield action, result

In [9]:
from collections import deque


def get_path_length_between_states(start_state, end_state):
    """bfs"""
    to_visit = deque([(start_state, 0)])
    visited = set()

    while to_visit:
        current_node, distance = to_visit.popleft()

        if current_node == end_state:
            return distance

        if current_node in visited:
            continue

        for n, _emit in get_possible_neighbors(current_node):
            to_visit.append((n, distance + 1))

        visited.add(current_node)

    return None


def get_path_between_states(start_state, end_state):
    """
    Performs BFS to find shortest path between states.
    Returns tuple of (path_length, path) if path exists, else (None, None)
    """
    to_visit = deque([(start_state, 0, [(None, None, start_state)])])
    visited = set()

    while to_visit:
        current_node, distance, path = to_visit.popleft()

        if current_node == end_state:
            return distance, path

        if current_node in visited:
            continue

        for action, (n, emit) in get_possible_neighbors(current_node):
            to_visit.append((n, distance + 1, path + [(action, current_node, n)]))

        visited.add(current_node)

    return None, None


In [10]:
# num_robo_hover_states = 2
# num_robo_hover_states = 3
# num_robo_hover_states = 4
# num_robo_hover_states = 5
num_robo_hover_states = 6
# num_robo_hover_states = 7
# num_robo_hover_states = 25

In [11]:
start_robos = tuple("A" for _ in range(num_robo_hover_states))
start_robos

('A', 'A', 'A', 'A', 'A', 'A')

In [12]:
get_path_between_states(("A", "3"), ("A", "5"))

(8,
 [(None, None, ('A', '3')),
  ('v', ('A', '3'), ('>', '3')),
  ('<', ('>', '3'), ('v', '3')),
  ('<', ('v', '3'), ('<', '3')),
  ('A', ('<', '3'), ('<', '2')),
  ('>', ('<', '2'), ('v', '2')),
  ('^', ('v', '2'), ('^', '2')),
  ('A', ('^', '2'), ('^', '5')),
  ('>', ('^', '5'), ('A', '5'))])

In [13]:
get_path_between_states(("A", "A", "3"), ("A", "A", "5"))

(20,
 [(None, None, ('A', 'A', '3')),
  ('v', ('A', 'A', '3'), ('>', 'A', '3')),
  ('<', ('>', 'A', '3'), ('v', 'A', '3')),
  ('A', ('v', 'A', '3'), ('v', '>', '3')),
  ('<', ('v', '>', '3'), ('<', '>', '3')),
  ('A', ('<', '>', '3'), ('<', 'v', '3')),
  ('A', ('<', 'v', '3'), ('<', '<', '3')),
  ('>', ('<', '<', '3'), ('v', '<', '3')),
  ('^', ('v', '<', '3'), ('^', '<', '3')),
  ('>', ('^', '<', '3'), ('A', '<', '3')),
  ('A', ('A', '<', '3'), ('A', '<', '2')),
  ('v', ('A', '<', '2'), ('>', '<', '2')),
  ('A', ('>', '<', '2'), ('>', 'v', '2')),
  ('^', ('>', 'v', '2'), ('A', 'v', '2')),
  ('<', ('A', 'v', '2'), ('^', 'v', '2')),
  ('A', ('^', 'v', '2'), ('^', '^', '2')),
  ('>', ('^', '^', '2'), ('A', '^', '2')),
  ('A', ('A', '^', '2'), ('A', '^', '5')),
  ('v', ('A', '^', '5'), ('>', '^', '5')),
  ('A', ('>', '^', '5'), ('>', 'A', '5')),
  ('^', ('>', 'A', '5'), ('A', 'A', '5'))])

In [14]:
get_path_between_states(("A", "A", "A", "3"), ("A", "A", "A", "5"))

(54,
 [(None, None, ('A', 'A', 'A', '3')),
  ('v', ('A', 'A', 'A', '3'), ('>', 'A', 'A', '3')),
  ('<', ('>', 'A', 'A', '3'), ('v', 'A', 'A', '3')),
  ('<', ('v', 'A', 'A', '3'), ('<', 'A', 'A', '3')),
  ('A', ('<', 'A', 'A', '3'), ('<', '^', 'A', '3')),
  ('>', ('<', '^', 'A', '3'), ('v', '^', 'A', '3')),
  ('A', ('v', '^', 'A', '3'), ('v', 'v', 'A', '3')),
  ('^', ('v', 'v', 'A', '3'), ('^', 'v', 'A', '3')),
  ('>', ('^', 'v', 'A', '3'), ('A', 'v', 'A', '3')),
  ('A', ('A', 'v', 'A', '3'), ('A', 'v', '>', '3')),
  ('v', ('A', 'v', '>', '3'), ('>', 'v', '>', '3')),
  ('<', ('>', 'v', '>', '3'), ('v', 'v', '>', '3')),
  ('<', ('v', 'v', '>', '3'), ('<', 'v', '>', '3')),
  ('A', ('<', 'v', '>', '3'), ('<', '<', '>', '3')),
  ('>', ('<', '<', '>', '3'), ('v', '<', '>', '3')),
  ('^', ('v', '<', '>', '3'), ('^', '<', '>', '3')),
  ('>', ('^', '<', '>', '3'), ('A', '<', '>', '3')),
  ('A', ('A', '<', '>', '3'), ('A', '<', 'v', '3')),
  ('A', ('A', '<', 'v', '3'), ('A', '<', '<', '3')),
  (

In [15]:
import itertools

list(itertools.permutations([1, 2, 3]))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [16]:
# ok here's how I can find the paths
# I look at manhattan distance between pairs of cells
# e.g. if it's (-1, 2) then I know that's one ['<', '^, '^']  then I just look at  all permutations of that which are permissible
# i.e. which don't pass thru Nothingness.

In [17]:
def manhattan_paths(array_2d: np.ndarray) -> dict:
    non_none_indices = np.argwhere(array_2d != None)
    values = array_2d[non_none_indices[:, 0], non_none_indices[:, 1]]

    elements = list(zip(values, non_none_indices[:, 0], non_none_indices[:, 1]))

    path_dict = {}

    for (val1, row1, col1), (val2, row2, col2) in itertools.product(elements, repeat=2):
        path = []

        row_distance = row2 - row1
        col_distance = col2 - col1

        if row_distance == 0:
            pass
        elif row_distance > 0:
            for _ in range(row_distance):
                path.append("v")
        else:
            for _ in range(abs(row_distance)):
                path.append("^")

        if col_distance == 0:
            pass
        elif col_distance > 0:
            for _ in range(col_distance):
                path.append(">")
        else:
            for _ in range(abs(col_distance)):
                path.append("<")

        path_dict[(val1, val2)] = path

    return path_dict

In [18]:
numeric_board

array([['7', '8', '9'],
       ['4', '5', '6'],
       ['1', '2', '3'],
       [None, '0', 'A']], dtype=object)

In [19]:
np.argwhere(numeric_board == "3")[0]

array([2, 2])

In [20]:
def check_path_legality(board, val_start, val_end, path):
    start = tuple(int(i) for i in np.argwhere(board == val_start)[0])
    end = tuple(int(i) for i in np.argwhere(board == val_end)[0])

    pos = start
    for p in path:
        new_pos = add_tuple(pos, dirs[p])

        if not in_bounds(new_pos, board.shape) or board[new_pos] is None:
            return False

        pos = new_pos

    assert pos == end

    return True

In [21]:
# check_dir_path_legality("<", "A", ["^", ">", ">"])
check_path_legality(directional_board, "<", "A", ["^", ">", ">"])

False

In [22]:
check_path_legality(directional_board, "<", "A", [">", ">", "^"])

True

In [23]:
check_path_legality(numeric_board, "0", "1", ["<", "^"])

False

In [24]:
check_path_legality(numeric_board, "0", "1", ["^", "<"])

True

In [25]:
manhattan_dir_paths = manhattan_paths(directional_board)
manhattan_dir_paths

{('^', '^'): [],
 ('^', 'A'): ['>'],
 ('^', '<'): ['v', '<'],
 ('^', 'v'): ['v'],
 ('^', '>'): ['v', '>'],
 ('A', '^'): ['<'],
 ('A', 'A'): [],
 ('A', '<'): ['v', '<', '<'],
 ('A', 'v'): ['v', '<'],
 ('A', '>'): ['v'],
 ('<', '^'): ['^', '>'],
 ('<', 'A'): ['^', '>', '>'],
 ('<', '<'): [],
 ('<', 'v'): ['>'],
 ('<', '>'): ['>', '>'],
 ('v', '^'): ['^'],
 ('v', 'A'): ['^', '>'],
 ('v', '<'): ['<'],
 ('v', 'v'): [],
 ('v', '>'): ['>'],
 ('>', '^'): ['^', '<'],
 ('>', 'A'): ['^'],
 ('>', '<'): ['<', '<'],
 ('>', 'v'): ['<'],
 ('>', '>'): []}

In [26]:
manhattan_num_paths = manhattan_paths(numeric_board)

In [27]:
def unique_permutations(lst):
    return [list(p) for p in set(itertools.permutations(lst))]


all_manhattan_dir_paths = {
    (start, end): [
        perm
        for perm in unique_permutations(path)
        if check_path_legality(directional_board, start, end, perm)
    ]
    for (start, end), path in manhattan_dir_paths.items()
}

all_manhattan_num_paths = {
    (start, end): [
        perm
        for perm in unique_permutations(path)
        if check_path_legality(numeric_board, start, end, perm)
    ]
    for (start, end), path in manhattan_num_paths.items()
}

In [28]:
all_manhattan_dir_paths

{('^', '^'): [[]],
 ('^', 'A'): [['>']],
 ('^', '<'): [['v', '<']],
 ('^', 'v'): [['v']],
 ('^', '>'): [['v', '>'], ['>', 'v']],
 ('A', '^'): [['<']],
 ('A', 'A'): [[]],
 ('A', '<'): [['<', 'v', '<'], ['v', '<', '<']],
 ('A', 'v'): [['<', 'v'], ['v', '<']],
 ('A', '>'): [['v']],
 ('<', '^'): [['>', '^']],
 ('<', 'A'): [['>', '>', '^'], ['>', '^', '>']],
 ('<', '<'): [[]],
 ('<', 'v'): [['>']],
 ('<', '>'): [['>', '>']],
 ('v', '^'): [['^']],
 ('v', 'A'): [['>', '^'], ['^', '>']],
 ('v', '<'): [['<']],
 ('v', 'v'): [[]],
 ('v', '>'): [['>']],
 ('>', '^'): [['^', '<'], ['<', '^']],
 ('>', 'A'): [['^']],
 ('>', '<'): [['<', '<']],
 ('>', 'v'): [['<']],
 ('>', '>'): [[]]}

In [29]:
all_manhattan_num_paths

{('7', '7'): [[]],
 ('7', '8'): [['>']],
 ('7', '9'): [['>', '>']],
 ('7', '4'): [['v']],
 ('7', '5'): [['v', '>'], ['>', 'v']],
 ('7', '6'): [['>', '>', 'v'], ['>', 'v', '>'], ['v', '>', '>']],
 ('7', '1'): [['v', 'v']],
 ('7', '2'): [['>', 'v', 'v'], ['v', 'v', '>'], ['v', '>', 'v']],
 ('7', '3'): [['>', 'v', '>', 'v'],
  ['>', '>', 'v', 'v'],
  ['>', 'v', 'v', '>'],
  ['v', '>', 'v', '>'],
  ['v', '>', '>', 'v'],
  ['v', 'v', '>', '>']],
 ('7', '0'): [['v', 'v', '>', 'v'],
  ['>', 'v', 'v', 'v'],
  ['v', '>', 'v', 'v']],
 ('7', 'A'): [['v', '>', 'v', '>', 'v'],
  ['>', 'v', 'v', '>', 'v'],
  ['>', 'v', '>', 'v', 'v'],
  ['v', '>', 'v', 'v', '>'],
  ['v', 'v', '>', '>', 'v'],
  ['>', 'v', 'v', 'v', '>'],
  ['>', '>', 'v', 'v', 'v'],
  ['v', 'v', '>', 'v', '>'],
  ['v', '>', '>', 'v', 'v']],
 ('8', '7'): [['<']],
 ('8', '8'): [[]],
 ('8', '9'): [['>']],
 ('8', '4'): [['<', 'v'], ['v', '<']],
 ('8', '5'): [['v']],
 ('8', '6'): [['v', '>'], ['>', 'v']],
 ('8', '1'): [['v', 'v', '<'], ['

In [30]:
from functools import lru_cache


@lru_cache(maxsize=100_000)
def aaa_directional_path(num_as, dir_p1, dir_p2):
    actions_paths = all_manhattan_dir_paths[dir_p1, dir_p2]

    if num_as == 0:
        # in this case the order does not matter; the lengths are the same
        # just take the first one
        return actions_paths[0]

    if num_as == -1:
        return []

    num_best_actions = np.inf
    best_actions = []
    for actions_path in actions_paths:
        path = ["A"] + actions_path + ["A"]
        rs = []
        actions = []
        for iter_num, (i, j) in enumerate(zip(path, path[1:]), start=1):
            r = aaa_directional_path(num_as - 1, i, j)
            rs.append(((i, j), r))

            actions.extend(r)
            if iter_num == len(path) - 1:
                # we don't add it in this case
                pass
            else:
                actions.append("A")

        if len(actions) < num_best_actions:
            best_actions = actions
            num_best_actions = len(actions)

    return best_actions


@lru_cache(maxsize=100_000)
def aaa_directional_distance(num_as, dir_p1, dir_p2):
    actions_paths = all_manhattan_dir_paths[dir_p1, dir_p2]

    if num_as == 0:
        # in this case the order does not matter; the lengths are the same
        # just take the first one
        return len(actions_paths[0])

    if num_as == -1:
        return 0

    num_best_actions = np.inf
    for actions_path in actions_paths:
        path = ["A"] + actions_path + ["A"]
        num_actions = 0
        for iter_num, (i, j) in enumerate(zip(path, path[1:]), start=1):
            num_lower_actions = aaa_directional_distance(num_as - 1, i, j)

            num_actions += num_lower_actions

            if iter_num == len(path) - 1:
                # we don't add it in this case
                pass
            else:
                num_actions += 1

        if num_actions < num_best_actions:
            num_best_actions = num_actions

    return num_best_actions

In [31]:
len(aaa_directional_path(2, ">", "<"))

18

In [32]:
get_path_between_states(("A", "A", ">", "0"), ("A", "A", "<", "0"))[0]

18

In [33]:
len(aaa_directional_path(3, ">", "<"))

46

In [34]:
get_path_between_states(("A", "A", "A", ">", "0"), ("A", "A", "A", "<", "0"))[0]

46

In [35]:
directional_action_cost = list(dirs.keys()) + ["A"]

for start, end, n in itertools.product(
    directional_action_cost, directional_action_cost, range(1, 7)
):
    start_full = tuple("A" for _ in range(n)) + (start, "0")
    end_full = tuple("A" for _ in range(n)) + (end, "0")

    # assert get_path_between_states(start_full, end_full)[0] == len(
    #     aaa_directional_distance(n, start, end)
    # )

    assert get_path_between_states(start_full, end_full)[0] == aaa_directional_distance(
        n, start, end
    )

In [36]:
len(aaa_directional_path(20, "<", ">"))

149983440

In [37]:
aaa_directional_distance(25, "<", ">")

14287938116

In [38]:
parsed_input

[['1', '4', '0', 'A'],
 ['1', '7', '0', 'A'],
 ['1', '6', '9', 'A'],
 ['8', '0', '3', 'A'],
 ['1', '2', '9', 'A']]

In [41]:
# NUM_AS = 2
NUM_AS = 25


def directional_action_cost(i, j):
    best_action_len = np.inf
    # best_action_path = []

    for num_action_path in all_manhattan_num_paths[i, j]:
        # print(num_action_path)
        sm = 0

        full_num_path = ["A"] + num_action_path + ["A"]
        # full_lower_path = []

        for dir_start, dir_end in zip(full_num_path, full_num_path[1:]):
            # n = aaa_directional_distance(NUM_AS - 1, dir_start, dir_end)
            # p = aaa_directional_path(NUM_AS - 1, dir_start, dir_end) + ["A"]
            # full_lower_path += p
            n = aaa_directional_distance(NUM_AS - 1, dir_start, dir_end) + 1
            sm += n

        if sm < best_action_len:
            best_action_len = sm
            # best_action_path = full_lower_path
            # print(
            #     f"Setting best action path to {best_action_len} for {(i, j)} with {best_action_path}"
            # )

    return best_action_len
    # return best_action_path


# for code in parsed_input[:3]:
total_complexity = 0
for code in parsed_input:
    print(code)
    numeric_code_path = ["A"] + code

    full_path_sm = 0
    for i, j in zip(numeric_code_path, numeric_code_path[1:]):
        # lower_action_path = directional_action_cost(i, j)
        # full_path_sm += len(lower_action_path)
        lower_action_cost = directional_action_cost(i, j)
        full_path_sm += lower_action_cost

    print(full_path_sm)
    num = int("".join(code[:-1]))
    complexity = full_path_sm * num
    total_complexity += complexity
    # print()

total_complexity

['1', '4', '0', 'A']
87513499934
['1', '7', '0', 'A']
87513499936
['1', '6', '9', 'A']
91059074548
['8', '0', '3', 'A']
93831469526
['1', '2', '9', 'A']
90594397580


129551515895690

In [46]:
# # start = "179A"

# start_state = ("A", "A", "A")
# actions = "<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A"
# # actions = "v<A<AA>^>vAA<^A>"
# print(start_state)

# current_state = start_state
# c = 0
# for a in actions:
#     c += 1
#     r = get_next_state(current_state, a)
#     if r:
#         current_state, emit = r
#         print(c, a, current_state)

#         if emit:
#             print(f"Emit {emit}")
#             c = 0
#     else:
#         print("!")